Import libraries

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
historical_features = pd.read_csv('historical_features.csv')

In [ ]:
X = historical_features.drop('label', axis=1)
y = historical_features['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.25, random_state=42)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
dval = xgb.DMatrix(X_val, label=y_val)

In [ ]:
param = {'max_depth': 3, 'eta': 1, 'objective': 'binary:logistic'}

In [ ]:
evallist = [(dtrain, 'train'), (dtest, 'eval')]

In [ ]:
num_round = 10
bst = xgb.train(param, dtrain, num_round, evallist)

In [ ]:
bst.save_model('0001.model')

In [ ]:
# bst = xgb.Booster({'nthread': 4})  # init model
# bst.load_model('model.bin')  # load model data

In [ ]:
xgb.plot_importance(bst)